# Portfolio Workflow

In [1]:
import logging
import logging.config

logging.config.fileConfig('./config/logging.ini')
logger = logging.getLogger('GenerateAIAlphaFactor')

import configparser
from platform import python_version
from pathlib import Path

# Set the import path for the project tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, 'tools')

# Project imports
import importlib
import trading_factors_yahoo as alpha_factors
importlib.reload(alpha_factors)
import utils
importlib.reload(utils)
import nonoverlapping_estimator as ai_estimator
importlib.reload(ai_estimator)

import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

logger.info(f'Python version: {python_version()}')
logger.info(f'Pandas version: {pd.__version__}')

2022-05-01 15:00:05,409|numexpr.utils|INFO|NumExpr defaulting to 4 threads.
Sci-Kit version: 0.24.1
Sci-Kit version: 0.24.1
2022-05-01 15:00:09,011|GenerateAIAlphaFactor|INFO|Python version: 3.8.8
2022-05-01 15:00:09,011|GenerateAIAlphaFactor|INFO|Pandas version: 1.3.5


In [20]:
config = configparser.ConfigParser()
config.read('./config/config.ini')
default_config = config['AIAlpha']

# Price History data and Alphs Factors

In [16]:
price_histories_file_name = default_config['DataDirectory'] + '/' + default_config['PriceHistoriesFileName']
logger.info(f'PRICE_HISTORIES_FILE|{price_histories_file_name}...')
price_histories = pd.read_csv(price_histories_file_name, header=[0, 1], index_col=[0], parse_dates=True, low_memory=False)
logger.info(f'PRICE_HISTORIES|{price_histories.index.min()}|{price_histories.index.max()}')
logger.info(f'Using {default_config["NumberOfYearsForAlpha"]} years of price history data to generate alpha factors.')
latest_date = price_histories.index.max() 
earliest_date = latest_date - pd.DateOffset(years=int(default_config["NumberOfYearsForAlpha"]))
price_histories = price_histories[(price_histories.index >= earliest_date) & (price_histories.index <= latest_date)]
logger.info(f'PRICE_HISTORIES_ALPHA|{price_histories.index.min()}|{price_histories.index.max()}')
close = price_histories.Close
logger.info(f'STOCK_TICKERS|{len(close.columns)}')
alpha_factors_file_name = default_config['DataDirectory'] + '/' + default_config['AlphaFactorsFileName']

logger.info(f'ALPHA_FACTORS_FILE|{alpha_factors_file_name}')
all_factors = pd.read_csv(alpha_factors_file_name, parse_dates=['Date']).set_index(['Date', 'Symbols']).sort_index()
logger.info('Alpha factors read.')

for alpha_factor in all_factors.columns:
    logger.info(f'ALPHA_FACTOR|{alpha_factor}')

2022-05-01 15:11:43,531|GenerateAIAlphaFactor|INFO|PRICE_HISTORIES_FILE|./data/price_histories_yahoo.csv...
2022-05-01 15:11:44,590|GenerateAIAlphaFactor|INFO|PRICE_HISTORIES|2017-05-01 00:00:00|2022-04-29 00:00:00
2022-05-01 15:11:44,591|GenerateAIAlphaFactor|INFO|Using 5 years of price history data to generate alpha factors.
2022-05-01 15:11:44,608|GenerateAIAlphaFactor|INFO|PRICE_HISTORIES_ALPHA|2017-05-01 00:00:00|2022-04-29 00:00:00
2022-05-01 15:11:44,614|GenerateAIAlphaFactor|INFO|STOCK_TICKERS|502
2022-05-01 15:11:44,615|GenerateAIAlphaFactor|INFO|ALPHA_FACTORS_FILE|./data/all_factors.csv
2022-05-01 15:11:45,462|GenerateAIAlphaFactor|INFO|Alpha factors read.
2022-05-01 15:11:45,462|GenerateAIAlphaFactor|INFO|ALPHA_FACTOR|momentum_252_day
2022-05-01 15:11:45,462|GenerateAIAlphaFactor|INFO|ALPHA_FACTOR|trailing_overnight_returns_10_day_smoothed
2022-05-01 15:11:45,462|GenerateAIAlphaFactor|INFO|ALPHA_FACTOR|mean_reversion_120_day_logret
2022-05-01 15:11:45,466|GenerateAIAlphaFact

# Stage 2b: Generate AI Alpha Factor

- Compute target values (y)
    - Quantize with 2 bins
- Train model for Feature importance
- Feature reduction
- Train model for AI Alpha Vector
- Compute AI Alpha Vectors for 1 year
- Save AI Alpha Vectors

## Compute the target values (y) and Shift back to create a 5 day forward prediciton

This is something you want to experiment with. If you are planning on holding on to assets for long periods of time, perhaps a 20, 40 or 60 forward prediciton will work better.

In [17]:
forward_prediciton_days = int(default_config['ForwardPredictionDays'])
prod_target_quantiles = int(default_config['PredictionQuantiles'])
prod_target_source = f'{forward_prediciton_days}Day{prod_target_quantiles}Quant'

logger.info(f'Setting {forward_prediciton_days} days-{prod_target_quantiles} quantiles to target {prod_target_source}')
all_assets = all_factors.index.levels[1].values.tolist()
logger.info(f'Factors from date: {all_factors.index.levels[0].min()} to date: {all_factors.index.levels[0].max()}')
features = all_factors.columns.tolist()

training_factors = pd.concat(
[
    all_factors,
    alpha_factors.FactorReturnQuantiles(price_histories, prod_target_quantiles, forward_prediciton_days).for_al(prod_target_source),
], axis=1).dropna()
training_factors.sort_index(inplace=True)

training_factors['target'] = training_factors.groupby(level=1)[prod_target_source].shift(-forward_prediciton_days)

logger.info(f'Creating training and label data...')
temp = training_factors.dropna().copy()
X = temp[features]
y = temp['target']
for feature in features:
    logger.info(f'TRAINING_FEATURE|{feature}')
logger.info(f'TRAINING_DATASET|{len(X)}|LABEL_DATASET|{len(y)}')

2022-05-01 15:11:48,244|GenerateAIAlphaFactor|INFO|Setting 5 days-2 quantiles to target 5Day2Quant
2022-05-01 15:11:48,246|GenerateAIAlphaFactor|INFO|Factors from date: 2018-05-01 00:00:00 to date: 2022-04-29 00:00:00
2022-05-01 15:12:12,055|GenerateAIAlphaFactor|INFO|Creating training and label data...
2022-05-01 15:12:12,129|GenerateAIAlphaFactor|INFO|TRAINING_FEATURE|momentum_252_day
2022-05-01 15:12:12,129|GenerateAIAlphaFactor|INFO|TRAINING_FEATURE|trailing_overnight_returns_10_day_smoothed
2022-05-01 15:12:12,129|GenerateAIAlphaFactor|INFO|TRAINING_FEATURE|mean_reversion_120_day_logret
2022-05-01 15:12:12,129|GenerateAIAlphaFactor|INFO|TRAINING_FEATURE|annualzed_volatility_20_day
2022-05-01 15:12:12,132|GenerateAIAlphaFactor|INFO|TRAINING_DATASET|503506|LABEL_DATASET|503506


In [ ]:
from sklearn.ensemble import RandomForestClassifier

n_days = 10
n_stocks = len(set(all_factors.index.get_level_values(level=1).values))
clf_random_state = 42

clf_parameters = {
    'criterion': 'entropy',
    'min_samples_leaf': n_days * n_stocks,
    'oob_score': True,
    'n_jobs': -1,
    'random_state': clf_random_state}

n_trees = int(default_config['RandomForestNTrees'])

logger.info(f'Creating RandomForestClassifier with {n_trees} trees...')
for key, value in clf_parameters.items():
    logger.info(f'Parameter: {key} set to {value}')
clf = RandomForestClassifier(n_trees, **clf_parameters)

logger.info(f'Creating Non-Overalpping Voter with {forward_prediciton_days-1} non-overlapping windows...')
clf_nov = ai_estimator.NoOverlapVoter(clf, n_skip_samples=forward_prediciton_days-1)

logger.info(f'Training classifier...')
clf_nov.fit(X, y)

logger.info(f'CLASSIFER|TRAIN|{clf_nov.score(X, y.values)}|OOB|{clf_nov.oob_score_}')

2022-05-01 15:15:52,082|GenerateAIAlphaFactor|INFO|Creating RandomForestClassifier with 10000 trees...
2022-05-01 15:15:52,083|GenerateAIAlphaFactor|INFO|Parameter: criterion set to entropy
2022-05-01 15:15:52,084|GenerateAIAlphaFactor|INFO|Parameter: min_samples_leaf set to 5020
2022-05-01 15:15:52,085|GenerateAIAlphaFactor|INFO|Parameter: oob_score set to True
2022-05-01 15:15:52,086|GenerateAIAlphaFactor|INFO|Parameter: n_jobs set to -1
2022-05-01 15:15:52,086|GenerateAIAlphaFactor|INFO|Parameter: random_state set to 42
2022-05-01 15:15:52,087|GenerateAIAlphaFactor|INFO|Creating Non-Overalpping Voter with 4 non-overlapping windows...
2022-05-01 15:15:52,089|GenerateAIAlphaFactor|INFO|Training classifier...


In [ ]:
ai_alpha_name = default_config['AIAlphaName']
factors_with_alpha = alpha_factors.add_alpha_score(training_factors[features].copy(), clf_nov, ai_alpha_name)

In [ ]:
factors_with_alpha

In [ ]:
factors_to_compare = features
alpha_factors.evaluate_ai_alpha(factors_with_alpha[factors_to_compare + [ai_alpha_name]], close)

In [ ]:
ai_alpha = factors_with_alpha[ai_alpha_name].copy()
alpha_vectors = ai_alpha.reset_index().pivot(index='Date', columns='Symbols', values=ai_alpha_name)
alpha_vectors.reset_index().to_csv('data/alpha_vectors.csv', index=False)